# LeetCode 175 — Combine Two Tables  
### A Complete Technical Notebook with Extended Theory, SQL Concepts, and Practical Examples

## Problem Statement

### Table: Person

| Column Name | Type    |
|------------|---------|
| personId   | int     |
| lastName   | varchar |
| firstName  | varchar |

**personId** is the primary key for this table.  
This table contains information about the ID of some persons and their first and last names.

### Table: Address

| Column Name | Type    |
|------------|---------|
| addressId  | int     |
| personId   | int     |
| city       | varchar |
| state      | varchar |

**addressId** is the primary key for this table.  
Each row contains information about the city and state of one person whose ID = personId.

---

## Task

Report the **first name**, **last name**, **city**, and **state** of each person in the Person table.  
If a person has no address in the Address table, return **NULL** for city and state.  
Return result in any order.

---

## Example Input

### Person table:

| personId | lastName | firstName |
|----------|----------|-----------|
| 1        | Wang     | Allen     |
| 2        | Alice    | Bob       |

### Address table:

| addressId | personId | city          | state      |
|-----------|----------|---------------|------------|
| 1         | 2        | New York City | New York   |
| 2         | 3        | Leetcode      | California |

### Output:

| firstName | lastName | city          | state    |
|-----------|----------|---------------|----------|
| Allen     | Wang     | NULL          | NULL     |
| Bob       | Alice    | New York City | New York |

---


# Entity-Relationship (ER) Diagram

Below is the ER diagram for the Person ↔ Address schema used in this LeetCode problem.

Because one person can have at most one address in this problem, this is a **One-to-Zero/One** relationship.



            ┌─────────────┐                  ┌──────────────┐
            │   Person    │ 1           0..1 │    Address    │
            ├─────────────┤                  ├───────────────┤
            │ personId PK │◄─────────────────┤ personId  FK  │
            │ firstName   │                  │ addressId  PK │
            │ lastName    │                  │ city          │
            └─────────────┘                  │ state         │
                                             └───────────────┘


In [1]:
# Environment setup: create in-memory SQLite DB, pandas, and helper show(query)
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")

def show(query: str) -> pd.DataFrame:
    """
    Execute a SQL query against the in-memory SQLite database and
    return a pandas DataFrame of the results for display.
    """
    df = pd.read_sql_query(query, conn)
    
    return display(df)



# Schema description

We model the exact LeetCode schema.

**Table: Person**
- `personId` INTEGER PRIMARY KEY  
- `lastName` TEXT  
- `firstName` TEXT

**Table: Address**
- `addressId` INTEGER PRIMARY KEY  
- `personId` INTEGER  — (conceptual FK referencing `Person.personId`)  
- `city` TEXT  
- `state` TEXT

Relationship: **Person (1) → Address (0..1)** in the problem statement (address may be missing). In some extended examples below we'll also create one-to-many variants to illustrate behavior.


In [2]:
# CREATE TABLE statements (SQLite)
create_statements = """
CREATE TABLE Person (
    personId INTEGER PRIMARY KEY,
    lastName TEXT,
    firstName TEXT
);

CREATE TABLE Address (
    addressId INTEGER PRIMARY KEY,
    personId INTEGER,
    city TEXT,
    state TEXT
);
"""
conn.executescript(create_statements)


# Sample data

We load the official LeetCode example rows plus a few additional rows that allow us to demonstrate other behaviors (e.g., missing addresses, addresses referencing non-existing person, and duplicates for extended lecture examples).

- Person rows: Allen Wang (1), Bob Alice (2)  
- Address rows: address for personId=2 and an address row for personId=3 (non-matching person in Person table to show orphan rows)


In [3]:
# INSERT statements: populate Person and Address with sample data
insert_statements = """
INSERT INTO Person (personId, lastName, firstName) VALUES
(1, 'Wang', 'Allen'),
(2, 'Alice', 'Bob');

INSERT INTO Address (addressId, personId, city, state) VALUES
(1, 2, 'New York City', 'New York'),
(2, 3, 'Leetcode', 'California'); -- personId 3 does not exist in Person
"""
conn.executescript(insert_statements)

# Quick check: show table contents
print("Person table:")
show("SELECT * FROM Person ORDER BY personId;")
print("\nAddress table:")
show("SELECT * FROM Address ORDER BY addressId;")

Person table:


,personId,lastName,firstName
0,1,Wang,Allen
1,2,Alice,Bob



Address table:


,addressId,personId,city,state
0,1,2,New York City,New York
1,2,3,Leetcode,California


# Problem requirements

Write a query that returns the columns:

- `firstName`  
- `lastName`  
- `city`  
- `state`

for **every person** in the `Person` table. If a person does not have an address row in `Address`, return `NULL` for `city` and `state`. Result order may be arbitrary.

---

Below begins an extended technical lecture built **around this exact problem and these tables**. The lecture explains SQL concepts thoroughly, shows multiple real-world scenarios, and demonstrates advanced techniques and patterns a technical lead would teach in a multi-hour session. Because we already created tables and loaded sample data, many examples below will reference these live tables.


# Lecture (Part 1) — Overview & Learning Goals

This problem looks tiny, but it's a perfect anchor for a long lecture that spans relational theory, practical SQL, data engineering patterns, performance, and ML/BI implications.

**Learning goals for this lecture:**

1. Master join semantics (INNER vs LEFT/RIGHT vs FULL) and why LEFT JOIN is the right answer here.  
2. Understand NULL semantics and how they affect queries and logic.  
3. Learn about cardinality & multiplicity and how joins can expand rows.  
4. Explore strategies to avoid row explosion and pick the “right” child row (window functions, aggregation).  
5. Cover schema design tradeoffs — normalization vs denormalization, sparse tables.  
6. Discuss indexes, explain plans, and performance for joins at scale.  
7. Practical case studies: ecommerce, banking/KYC, healthcare, telemetry — how this pattern appears in each.  
8. ETL / ML considerations: feature generation, missingness, reproducibility.  
9. Anti-patterns, debugging steps, and interview-level questions derived from the problem.  

We will alternate conceptual explanations with runnable SQL examples (using the tables already created) so you can see behaviors directly.


# Lecture (Part 2) — Keys, Relationships, and the Data Model

**Primary Key (PK)**  
- Uniquely identifies each row in a table. `Person.personId` and `Address.addressId` are PKs.  
- PKs support integrity and indexing choices; they make joins predictable.

**Foreign Key (FK)**  
- `Address.personId` conceptually references `Person.personId`. In production RDBMS you would declare `FOREIGN KEY (personId) REFERENCES Person(personId)` to enforce referential integrity. SQLite can support FK constraints but they must be enabled explicitly; for this notebook we keep FK conceptual to better show some edge behaviors (like orphan addresses).

**Relationship types** (with examples tied to our problem):
- **1:0..1 (one-to-zero-or-one)** — Person → Address (a person may have zero or one address). This implies LEFT JOIN from Person to Address to keep all Persons.
- **1:Many** — E.g., Customer → Orders. If Address were multiple per person (historical addresses), join results would have multiple rows per person.
- **Many:Many** — e.g., Student ↔ Course via Enrollment table. Different modeling approach (bridge table).

**Why modeling matters**: Choosing normalized vs denormalized modeling affects query complexity, storage, write patterns, and the join logic we use in queries.


# Lecture (Part 3) — Join types and formal semantics

**INNER JOIN**  
- Returns only rows where join condition matches both sides.  
- If we used `INNER JOIN` here, persons without an address would be omitted — wrong for the problem.

**LEFT OUTER JOIN (LEFT JOIN)**  
- Returns all rows from left table (Person) and matching rows from right table (Address). When no match, right-side columns are NULL.  
- This is the correct pattern for "show all persons and address info when present".

**RIGHT OUTER JOIN** / **FULL OUTER JOIN**  
- RIGHT preserves right table; FULL preserves both. SQLite doesn't natively support RIGHT/FULL but many DBs do. They are useful for reconciliation tasks (show rows unique to either side).

**CROSS JOIN**  
- Cartesian product — rarely used unless explicitly combining all pairs.

**JOIN predicate positioning**  
- Write `ON` for join condition and `WHERE` for filter. Beware: `WHERE` applied after join will filter out NULLs created by LEFT JOIN — e.g., `WHERE a.city = 'X'` after LEFT JOIN undo the outer join effect for persons without addresses. Use `AND (a.city = 'X' OR a.city IS NULL)` if your intent is to preserve NULL rows.

**Example** (why LEFT JOIN is correct) — run the core LEFT JOIN on our live tables (we will run the final solution later). For now, conceptual understanding: Person 1 has no address, so LEFT JOIN ensures it appears with city/state = NULL.


# Join Visualizations

SQL joins can be visualized using basic set diagrams. Below are the join diagrams relevant to this problem.

---

## LEFT JOIN (Used in this LeetCode Problem)

Retrieve **all Person rows**, and attach Address data *when available*.  
If no match exists, Address columns become **NULL**.



                LEFT JOIN (People-centered)
        ┌─────────────┐
        │  Person     │──────────────┐
        └─────────────┘              │
                                     ▼
     ┌─────────────────────────────────────────────────┐
     │   All rows from Person + matching Address rows   │
     └─────────────────────────────────────────────────┘
             ▲                     ▲
             │                     │
       ┌──────────┐         ┌───────────┐
       │ Person    │         │ Address   │
       └──────────┘         └───────────┘

Rows from Person with no matching address → Address columns become NULL.


## INNER JOIN

Only return rows that appear in **both** Person AND Address.

                  INNER JOIN
     ┌─────────────┐   ┌──────────────┐
     │   Person    │ ∩ │   Address     │
     └─────────────┘   └──────────────┘

Any missing address or person row is excluded.


## RIGHT JOIN (not in SQLite, but conceptually useful)

Return all rows from **Address**, with Person columns NULL when no match.

                  RIGHT JOIN
       ┌───────────────┐
       │    Address     │───────────────┐
       └───────────────┘               │
                                        ▼
          ┌────────────────────────────────────────────┐
          │ All rows from Address + matching Person     │
          └────────────────────────────────────────────┘


## FULL OUTER JOIN (not in SQLite, but important concept)

Return all Person rows + all Address rows.

                  FULL OUTER JOIN
         ┌──────────────┐   ┌───────────────┐
         │   Person     │ ∪ │    Address     │
         └──────────────┘   └───────────────┘

Missing matches on either side → NULLs appear.


# Lecture (Part 4) — NULL semantics and three-valued logic

**NULL is not a value; it's an absence of a value (unknown/missing).** Important consequences:

- `NULL = NULL` yields `UNKNOWN`, *not* true. Use `IS NULL` / `IS NOT NULL`.  
- Filters like `WHERE a.city = 'New York'` exclude rows where `a.city` is NULL. After LEFT JOIN, that will remove persons without addresses if not careful.  
- Aggregation: `COUNT(column)` counts non-NULL values, `COUNT(*)` counts rows.  
- Use `COALESCE(col, default)` to replace NULL for presentation or downstream systems.

**Practical guidance:**
- Preserve NULLs during data collection; decide later whether to impute or label them.  
- Use `COALESCE` or conditional expressions when preparing BI-friendly output.  
- For boolean logic, remember the three-valued logic: TRUE / FALSE / UNKNOWN — plan filters accordingly.


# Lecture (Part 5) — Cardinality & multiplicative joins (why rows can explode)

If the right table has multiple matching rows for a left-key, the join will produce multiple rows per left key (one per matching child). Example: if `Address` had three addresses for `personId = 2`, a plain LEFT JOIN will return three rows for that person.

**Why that matters:**
- Reporting: you might expect one row per person but get many — aggregated metrics become inflated.
- ML feature engineering: if you join transactions directly, you might duplicate user rows unless you aggregate transactions first.

**Solution patterns:**
1. **Choose and join a single child row** — e.g., latest address using `ROW_NUMBER()` window function or pre-aggregate that chooses a single row.  
2. **Pre-aggregate the child table** (e.g., count of addresses per person) and join the aggregate (one row per person).  
3. **Use EXISTS** for membership queries (returns a boolean, no duplication).  
4. **Group and use MAX/MIN** to get a representative child (careful with ties).

We'll demonstrate all these patterns later with runnable queries using extra demo tables.


# Lecture (Part 6) — Practical SQL patterns: window functions, pre-aggregation, EXISTS

**Window functions** (`ROW_NUMBER()`, `RANK()`, `FIRST_VALUE()`) are idiomatic when you need "the latest/first" child row per parent.

**Pre-aggregation** (subquery or CTE with `GROUP BY`) is better when you need aggregated metrics (counts, sums) from children.

**EXISTS** is efficient when you only need to know if any child exists — avoids joining full child rows and avoids duplication.

**Example intentions:**
- "List persons with their latest address" → window function (ROW_NUMBER()) or pre-aggregate by MAX(date)+tie-breaker.
- "List persons and number of addresses" → GROUP BY on Address then LEFT JOIN aggregated result.
- "List persons who have an address" → EXISTS or INNER JOIN depending on whether you want persons without addresses excluded.


# Lecture (Part 7) — Schema design tradeoffs: normalization vs denormalization

**Normalized design** (one row per entity and related child tables) is the default for OLTP:
- Minimal redundancy, easy updates, consistent writes.
- Query cost increases with joins.

**Denormalization**:
- Duplicate certain child attributes into parent (e.g., store `city` in `Person`) to speed reads.
- Useful in read-heavy workloads (dashboards, APIs) but increases write complexity and risk of inconsistent data.

**Hybrid**:
- Keep normalized canonical data; maintain denormalized summary tables or materialized views for analytics and low-latency reads.
- Use scheduled or incremental ETL to refresh summary tables.

**When to denormalize in practice**:
- When >95% of queries require joined data and the join cost is high.
- When the child attributes are small and update rate is low, making cost of maintaining denormalized columns manageable.


# Lecture (Part 8) — Indexing, query planning & performance considerations

**Indexes for joins**:
- Join on PK and FK — ensure FK side is indexed for best performance.
- On large tables, missing indexes can make joins explode to full-table scans.

**Explain plans**:
- Use `EXPLAIN` / `EXPLAIN QUERY PLAN` to view how the engine plans the join.
- The planner decides join order and algorithm (nested loop, hash join — engine dependent).

**Analytics scale**:
- In OLAP systems (Redshift, BigQuery, Snowflake), joins between massive tables are expensive. Use partitioning, clustering, or pre-aggregated tables.

**Practical rule-of-thumb**:
- For OLTP schemas with indexed PK/FK, joins on IDs are fast. For analytics, consider materialized views or ETL to precompute heavy joins.


# Lecture (Part 9) — Real-world case studies (how this problem pattern appears in production)

**Case study: Ecommerce (Customer & Shipping Address)**
- Customer master table, optional shipping address table (user may not have provided address).
- For checkout: you need an address — if missing, prompt user; for analytics, you may show user with NULL address.
- If multiple addresses per customer (billing, shipping), you must pick an appropriate address via `address_type` or pick latest.

**Case study: Banking (Customer & KYC address)**
- Regulatory requirement: maintain current KYC address and previous addresses.
- Query to list active customers and their current KYC address uses window function to pick latest `effective_date` or `is_current` flag.

**Case study: Healthcare (Patient & Contact info)**
- Patients may have multiple contact points; when generating a contact list, we pick primary contact (is_primary flag).
- Incorrect joins could lead to duplicates in patient counts for epidemiological metrics.

**Case study: AdTech (User & Device / Events)**
- High-volume events table joined to user table — never join raw events directly to models; aggregate events (session counts, last_seen) in a feature store and join aggregated features to user master.


# Lecture (Part 10) — ETL / ML implications: feature generation & missingness

**From relational join to feature table**:
- Feature table: one row per entity (e.g., user) with fixed number of features. Joining many sources must result in single row per key.
- Missingness (NULL) can be informative — e.g., no address may indicate incomplete onboarding.

**Feature engineering patterns**
- Pre-aggregate event logs by user: counts, recency, categorical encodings.
- Join master profile (Person) with aggregated features using LEFT JOIN so all users are present.
- After joining, apply imputation strategy: domain defaults, learned imputation, or sentinel categories.

**Reproducibility**
- Keep timestamps and run joins with strict snapshot semantics (i.e., specify as-of dates) to make training data reproducible.


# Lecture (Part 11) — Common mistakes, anti-patterns, and debugging checklist

**Common mistakes**
- Using INNER JOIN when wanting to preserve master rows → silent data loss.  
- Joining on non-unique keys accidentally → multiplicative rows.  
- Post-join filtering in WHERE that nullifies the LEFT JOIN.  
- Assuming ordering without `ORDER BY` in window function tie-breakers.

**Debugging checklist**
1. Confirm expected relationship cardinality (1:1, 1:N).  
2. Check uniqueness of join keys on both sides (`SELECT personId, COUNT(*) FROM Address GROUP BY personId HAVING COUNT(*)>1`).  
3. Run small queries to inspect raw joined output.  
4. Use `EXPLAIN` to check execution plan and index usage.  
5. If counts mismatch, compare counts before and after join and use `DISTINCT` to inspect duplicates.


# Lecture (Part 12) — Equivalent in pandas (brief comparison)

- `pd.merge(left, right, how='left', on='personId')` ≈ SQL `LEFT JOIN`.
- Pandas merges are in-memory and may blow memory for large datasets; RDBMS handle larger-than-memory and provide optimized join algorithms.
- Use `groupby().agg()` in pandas to pre-aggregate before merge to avoid duplication.

**Key conceptual parity**: the join semantics, NULL handling (NaN in pandas), and aggregation strategies are the same; implementation differences (memory, performance) matter in production.


# Lecture (Part 13) — Hands-on demonstrations & extended SQL exercises (we will now run queries on live tables)

We already have `Person` and `Address` loaded. We'll demonstrate:

1. Why `INNER JOIN` is wrong for this problem.  
2. The correct `LEFT JOIN` solution (final).  
3. How orphan rows in Address behave (address rows without matching Person).  
4. Using `COALESCE` for presentation.  
5. Simulating many addresses per person and showing window functions and pre-aggregation.  
6. Using `EXISTS` for boolean membership checks.


In [4]:
# 1) Show INNER JOIN behavior (demonstrate data loss)
print("INNER JOIN (drops persons without address):")
show("""
SELECT p.personId, p.firstName, p.lastName, a.city, a.state
FROM Person p
INNER JOIN Address a ON p.personId = a.personId
ORDER BY p.personId;
""")


INNER JOIN (drops persons without address):


,personId,firstName,lastName,city,state
0,2,Bob,Alice,New York City,New York


In [5]:
# 2) Correct LEFT JOIN solution (core answer)
print("LEFT JOIN (keeps all persons with NULL for missing addresses):")
show("""
SELECT p.firstName, p.lastName, a.city, a.state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personId
ORDER BY p.personId;
""")


LEFT JOIN (keeps all persons with NULL for missing addresses):


,firstName,lastName,city,state
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


In [6]:
# 3) Show orphan address rows (addresses without matching person)
print("Orphan addresses (exist in Address but no matching Person):")
show("""
SELECT a.*
FROM Address a
LEFT JOIN Person p ON a.personId = p.personId
WHERE p.personId IS NULL;
""")


Orphan addresses (exist in Address but no matching Person):


,addressId,personId,city,state
0,2,3,Leetcode,California


In [7]:
# 4) COALESCE for presentation: replace NULL with 'Unknown'
print("Presentation-friendly output (COALESCE):")
show("""
SELECT p.firstName, p.lastName,
       COALESCE(a.city, 'Unknown') AS city,
       COALESCE(a.state, 'Unknown') AS state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personId
ORDER BY p.personId;
""")


Presentation-friendly output (COALESCE):


,firstName,lastName,city,state
0,Allen,Wang,Unknown,Unknown
1,Bob,Alice,New York City,New York


# Lecture (Part 14) — Demonstrate one-to-many scenario and window function solution

We will create additional demo tables to show the one-to-many case where a person has multiple addresses (historical), then show how to pick the latest address per person using window functions and pre-aggregation. These are common real-world extensions of this problem.


In [8]:
# Create demo tables to simulate multiple addresses per person
conn.executescript("""
DROP TABLE IF EXISTS PersonDemo;
DROP TABLE IF EXISTS AddressDemo;

CREATE TABLE PersonDemo (personId INTEGER PRIMARY KEY, firstName TEXT, lastName TEXT);
CREATE TABLE AddressDemo (addressId INTEGER PRIMARY KEY, personId INTEGER, city TEXT, state TEXT, created_ts TEXT);

INSERT INTO PersonDemo (personId, firstName, lastName) VALUES
(1, 'Allen', 'Wang'),
(2, 'Bob', 'Alice'),
(3, 'Cara', 'Doe');

INSERT INTO AddressDemo (addressId, personId, city, state, created_ts) VALUES
(1, 2, 'New York City', 'New York', '2020-01-01'),
(2, 3, 'Leetcode', 'California', '2021-05-01'),
(3, 2, 'Brooklyn', 'New York', '2022-03-15'),
(4, 2, 'Queens', 'New York', '2023-07-20'); -- multiple addresses for personId=2
""")

print("PersonDemo:")
show("SELECT * FROM PersonDemo ORDER BY personId;")
print("\nAddressDemo:")
show("SELECT * FROM AddressDemo ORDER BY personId, addressId;")


PersonDemo:


,personId,firstName,lastName
0,1,Allen,Wang
1,2,Bob,Alice
2,3,Cara,Doe



AddressDemo:


,addressId,personId,city,state,created_ts
0,1,2,New York City,New York,2020-01-01
1,3,2,Brooklyn,New York,2022-03-15
2,4,2,Queens,New York,2023-07-20
3,2,3,Leetcode,California,2021-05-01


## Problem: naive LEFT JOIN duplicates master rows when multiple children exist

Run naive LEFT JOIN and observe multiplicity.

In [9]:
show("""
SELECT p.personId, p.firstName, p.lastName, a.addressId, a.city, a.created_ts
FROM PersonDemo p
LEFT JOIN AddressDemo a ON p.personId = a.personId
ORDER BY p.personId, a.addressId;
""")


,personId,firstName,lastName,addressId,city,created_ts
0,1,Allen,Wang,NaN,None,None
1,2,Bob,Alice,1.0,New York City,2020-01-01
2,2,Bob,Alice,3.0,Brooklyn,2022-03-15
3,2,Bob,Alice,4.0,Queens,2023-07-20
4,3,Cara,Doe,2.0,Leetcode,2021-05-01


## Solution A: Use ROW_NUMBER() to pick the latest address per person (window function)

This pattern yields exactly one child row per parent (or NULL if none).


In [10]:
show("""
WITH ranked_addresses AS (
  SELECT a.*,
         ROW_NUMBER() OVER (PARTITION BY a.personId ORDER BY datetime(a.created_ts) DESC, a.addressId DESC) AS rn
  FROM AddressDemo a
)
SELECT p.personId, p.firstName, p.lastName, ra.addressId, ra.city, ra.created_ts
FROM PersonDemo p
LEFT JOIN ranked_addresses ra ON p.personId = ra.personId AND ra.rn = 1
ORDER BY p.personId;
""")


,personId,firstName,lastName,addressId,city,created_ts
0,1,Allen,Wang,NaN,None,None
1,2,Bob,Alice,4.0,Queens,2023-07-20
2,3,Cara,Doe,2.0,Leetcode,2021-05-01


## Solution B: Pre-aggregate to get latest timestamp then join to child table

This is another common pattern; it avoids window functions on some engines or leverages indexes on timestamp columns.


In [11]:
show("""
WITH latest AS (
  SELECT personId, MAX(created_ts) AS max_ts
  FROM AddressDemo
  GROUP BY personId
)
SELECT p.personId, p.firstName, p.lastName, a.addressId, a.city, a.created_ts
FROM PersonDemo p
LEFT JOIN latest l ON p.personId = l.personId
LEFT JOIN AddressDemo a ON a.personId = l.personId AND a.created_ts = l.max_ts
ORDER BY p.personId;
""")


,personId,firstName,lastName,addressId,city,created_ts
0,1,Allen,Wang,NaN,None,None
1,2,Bob,Alice,4.0,Queens,2023-07-20
2,3,Cara,Doe,2.0,Leetcode,2021-05-01


# Lecture (Part 15) — EXISTS vs JOIN for membership checks

If you only want to know whether a person has an address (boolean), `EXISTS` is preferable — efficient and avoids duplication.


In [12]:
show("""
SELECT p.personId, p.firstName, p.lastName,
       CASE WHEN EXISTS (SELECT 1 FROM AddressDemo a WHERE a.personId = p.personId) THEN 1 ELSE 0 END AS has_address
FROM PersonDemo p
ORDER BY p.personId;
""")


,personId,firstName,lastName,has_address
0,1,Allen,Wang,0
1,2,Bob,Alice,1
2,3,Cara,Doe,1


# Lecture (Part 16) — Indexing and EXPLAIN (conceptual + small demonstration)

In production, you'd add an index on `Address.personId`. In SQLite we can create it and inspect EXPLAIN QUERY PLAN to see its effect (SQLite's planner output is basic but illustrative). We'll create an index and show plans for a join query before and after.


In [13]:
# Show explain plan before index
print("EXPLAIN QUERY PLAN (before index):")
show("EXPLAIN QUERY PLAN SELECT p.personId FROM PersonDemo p LEFT JOIN AddressDemo a ON p.personId = a.personId;")

# Create index on AddressDemo.personId
conn.execute("CREATE INDEX IF NOT EXISTS idx_addressdemo_personId ON AddressDemo(personId);")

# Show explain plan after index
print("\nEXPLAIN QUERY PLAN (after index):")
show("EXPLAIN QUERY PLAN SELECT p.personId FROM PersonDemo p LEFT JOIN AddressDemo a ON p.personId = a.personId;")


EXPLAIN QUERY PLAN (before index):


,id,parent,notused,detail
0,3,0,0,SCAN p
1,7,0,0,BLOOM FILTER ON a (personId=?)
2,16,0,0,SEARCH a USING AUTOMATIC COVERING INDEX (perso...



EXPLAIN QUERY PLAN (after index):


,id,parent,notused,detail
0,3,0,0,SCAN p
1,5,0,0,SEARCH a USING COVERING INDEX idx_addressdemo_...


# Lecture (Part 17) — Production considerations: ACID, FK constraints, and data quality

**Foreign Keys & ACID**  
- In transactional systems, enforce FK constraints to prevent orphan child records.  
- Use transactions for multi-table updates to keep consistency.

**Data quality & monitoring**  
- Monitor orphan counts: `SELECT COUNT(*) FROM Address a LEFT JOIN Person p ON a.personId = p.personId WHERE p.personId IS NULL`.  
- Set up alerts or ETL rules to correct or reconcile orphans.

**Change data capture & slowly changing dimensions**  
- For historical addresses, keep history with `effective_from`/`effective_to` or a `is_current` flag.  
- For analytics, pick addresses as-of a snapshot date to ensure reproducible metrics.


# Step-by-step reasoning for the LeetCode problem (concise, applied to our tables)

1. The problem requires **every person** to appear in the output. That makes `Person` the master table — it must be the left side of a LEFT JOIN.  
2. We only need to pull `firstName`, `lastName` from `Person` and `city`, `state` from `Address`.  
3. If there's no matching `Address` row, `city` and `state` must be `NULL`. This is exactly what LEFT JOIN provides.  
4. Therefore the final SQL is:

```sql
SELECT p.firstName, p.lastName, a.city, a.state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personId;


# Output verification

Expected result (based on sample data we inserted):

| firstName | lastName | city          | state    |
|-----------|----------|---------------|----------|
| Allen     | Wang     | NULL          | NULL     |
| Bob       | Alice    | New York City | New York |

Note: There's an orphan address row in `Address` for `personId = 3` which is not in `Person` — it is ignored by the LEFT JOIN (unless you start from Address).


In [14]:
# Final SQL query used for verification
final_query = """
SELECT 
    p.firstName,
    p.lastName,
    a.city,
    a.state
FROM Person p
LEFT JOIN Address a
    ON p.personId = a.personId;
"""

In [15]:
# show() output: display the results using the helper function
show(final_query)

,firstName,lastName,city,state
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


# Summary & Final Takeaways (technical lead level)

- **Core solution**: `LEFT JOIN` from `Person` to `Address` returns the required output and preserves persons without addresses.  
- **Why this problem matters**: It's a compact demonstration of join semantics, NULL handling, cardinality, schema design tradeoffs, and practical ETL/ML implications.  
- **Key technical lessons**:
  - Identify master table and choose join direction accordingly.  
  - Understand NULLs — how they behave in joins and filters.  
  - Be aware of multiplicative joins — use window functions, pre-aggregation, or EXISTS as appropriate.  
  - Use indexing and examine query plans for performance.  
  - For production, enforce referential integrity, monitor orphans, and design for reproducible joins (snapshotting, as-of timestamps).  
- **Interview & engineering depth**: This LeetCode problem can be extended into advanced interview questions: handle many addresses per person, choose latest address, ensure deterministic tie-breaking, design denormalized views for BI, and reason about costs at scale.

---

If you'd like, I can:
- Export this entire notebook as a `.ipynb` file for download.  
- Add diagrams (ER diagram + join-visuals).  
- Add graded exercises, quiz questions, and answers based on this lecture.
